# Tune the best ML model for ICDHyr

The best = the one with the lowest average MAPE score from "models_filter".

Preprocessing and feature selection is performed after splitting the data (therefore, only the samples in the training data are available for these steps). 

In [1]:
#import libraries
import pandas as pd
import numpy as np
from IPython.display import display

from functools import partial #to pass parameters to function inside another function
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import linear_model
from sklearn import svm

In [2]:
#import self-made functions
%run /Users/mariekececilia/Documents/master_thesis_code/methods.ipynb

In [3]:
#set a random seed to make reproducible results (used in mutual information)
seed = 0

## Load and clean data

In [4]:
#load transcriptional and fluxomic data
gexp, flux = load_gerosa()

#clean the transcriptional data (remove duplicates)
gexp, groups = clean_gexp_g(gexp)

#define features
X = gexp

#define targets and extract their fluxes
target_labels = ['R_ICDHyr']
targets = dict()
for target_label in target_labels:
    targets[target_label] = flux[target_label]

## Create models/pipelines

Choose selection parameters:

In [5]:
cov_filters = [0.2] #same as in the best model
k_numbers = [15, 20, 30] #same k as in the best model + 1 larger and 1 smaller k

Choose learning algorithms:

In [6]:
algorithms = dict({
    'lr': linear_model.LinearRegression(),
    'lasso_a1': linear_model.Lasso(alpha = 1),
    'lasso_a0.1': linear_model.Lasso(alpha = 0.1),
    'lasso_a0.05': linear_model.Lasso(alpha = 0.05), 
    'lasso_a0.001': linear_model.Lasso(alpha = 0.001),
    'ridge_a1': linear_model.Ridge(alpha = 1),
    'ridge_a0.1': linear_model.Ridge(alpha = 0.1),
    'ridge_a0.05': linear_model.Ridge(alpha = 0.05),
    'ridge_a0.001': linear_model.Ridge(alpha = 0.001),
    })

Make combinations:

In [7]:
def get_pipelines(algorithm, cov_filters, k_numbers):
    models = dict()

    for cov_filter in cov_filters:  
        
        for k in k_numbers:
            #selection by mi – same as in the best model (skipping the other scoring method)
            steps = [
                ('filter', CoVSelector(p = cov_filter)),
                ('selection', SelectKBest(score_func = partial(
                    mutual_info_regression, 
                    random_state=seed), 
                                          k = k)), 
                ('standardize', StandardScaler()),
                ('m', algorithm)
            ]
            models['v%.0f_mi%.0f' % (cov_filter*100, k)] = Pipeline(steps=steps)

    return models

In [8]:
models = dict()

for name, algorithm in algorithms.items():
     models[name]= get_pipelines(algorithm, cov_filters = cov_filters, k_numbers = k_numbers)

## Learning

In [9]:
# evaluate the models and store results in nested dicts: target -> learning algorithm -> scores/preds of all selection combinations
scores = dict()
preds = dict()
for y_name, y in targets.items():
    model_scores = dict()
    model_preds = dict()
    for model_name, pipelines in models.items():
        #get scores
        model_scores[model_name] = compare_models(pipelines, X, y)
        #get predictions
        model_preds[model_name] = get_models_preds(pipelines, X, y)
    scores[y_name] = model_scores
    preds[y_name] = model_preds

/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.800e-03, tolerance: 1.493e-03
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 6.891e-03, tolerance: 3.717e-03
  model = cd_fast.enet_coordinate_descent(
/Users/mariekececilia/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consi

### Save data to use in other notebooks

Scores and predictions are visualized in the Notebook 'compare_models', which also merges the results with the results from all other models built.

In [10]:
scores_tune_icdhyr = scores
preds_tune_icdhyr = preds
%store scores_tune_icdhyr
%store preds_tune_icdhyr

Stored 'scores_tune_icdhyr' (dict)
Stored 'preds_tune_icdhyr' (dict)


### Scores and predictions

In [11]:
for y in targets.keys():
    for algorithm in algorithms.keys():
        print()
        print(y + '\t(target)') 
        print(algorithm, '\t(learning algorithm)') 
        print('–> summary of all cv split scores for each combination of selection methods:')
        df = scores[y][algorithm][1].sort_values(by = 'average')
        df.index.name = 'Selection combo'
        df.columns.name = 'Score'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all individual cv-split scores for each combination of selection methods:')
        df = scores[y][algorithm][0]
        df.columns.name = 'Selection combo'
        display(df)
        
        print()
        print(y + '\t(target)') 
        print(algorithm + '\t(learning algorithm)') 
        print('–> all predictions for each combination of selection methods (+ actual values):')
        df = preds[y][algorithm]
        df.index.name = 'Test set'
        df.columns.name = 'Selection combo'
        display(df)


R_ICDHyr	(target)
lr 	(learning algorithm)
–> summary of all cv split scores for each combination of selection methods:


Score,average,std
Selection combo,,
v20_mi15,0.281726,0.361808
v20_mi30,0.289191,0.163812
v20_mi20,0.359214,0.309566



R_ICDHyr	(target)
lr	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v20_mi15,v20_mi20,v20_mi30
Test set,,,
Acetate,0.141819,0.075580,0.204127
Fructose,1.176498,0.557744,0.235388
Galactose,0.058176,1.069707,0.671772
Glucose,0.035424,0.193387,0.192835
Glycerol,0.066551,0.063733,0.143427
Gluconate,0.335965,0.358364,0.340930
Pyruvate,0.392126,0.360838,0.371025
Succinate,0.047244,0.194357,0.154021



R_ICDHyr	(target)
lr	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


...

Score,average,std
Selection combo,,
v20_mi15,0.280201,0.361073
v20_mi30,0.289241,0.164017
v20_mi20,0.359417,0.310183



R_ICDHyr	(target)
ridge_a0.001	(learning algorithm)
–> all individual cv-split scores for each combination of selection methods:


Selection combo,v20_mi15,v20_mi20,v20_mi30
Test set,,,
Acetate,0.141444,0.075595,0.203787
Fructose,1.172038,0.556523,0.235366
Galactose,0.049399,1.072226,0.672371
Glucose,0.035661,0.193378,0.192824
Glycerol,0.066569,0.063733,0.143391
Gluconate,0.336329,0.358545,0.341085
Pyruvate,0.392245,0.360867,0.371035
Succinate,0.047927,0.194472,0.154068



R_ICDHyr	(target)
ridge_a0.001	(learning algorithm)
–> all predictions for each combination of selection methods (+ actual values):


Selection combo,actual,v20_mi15,v20_mi20,v20_mi30
Test set,,,,
Acetate,4.696417,4.032136,5.051443,5.653485
Fructose,4.566119,9.917782,7.107268,5.640828
Galactose,0.496177,0.471667,1.028192,0.829793
Glucose,2.977971,2.871773,2.402097,2.403748
Glycerol,2.464836,2.300756,2.307746,2.111401
Gluconate,1.154034,1.542168,1.567807,1.547657
Pyruvate,7.979320,4.849470,5.099847,5.018716
Succinate,3.038949,2.893302,2.447959,2.570744
